# Zad 1.  
Zbadamy ewolucję układu na sieci kwadratowej o rozmiarze 100 ×100 dla początkowego losowego rozkładu dwóch opinii w układzie. Sporządzamy również wykres liczby opinii na ‘tak’ od czasu. Przyjmijmy sekwencyjny schemat aktualizacji opinii.  

In [ ]:
# packages required
using Random, Plots

function voter_model(; L=100, max_iter=100_000)

    network = rand(0:1, (L, L)) # L x L randomized binary matrix {1: "yes", 0: "no")
    i = 0                       # iteration counter
    yesses = []                 # number of ones/yes answers container

    # model loop
    start_time = time()
    while (0 < sum(network) < L^2) && (i < max_iter)
    # sum(network) de facto means number of ones/yes answers,
    # having sum(network)==0 || sum(network)==L^2 means that model converged
        i += 1
        # foreach node in the network, copy opinion from one of its neighbout
        for x in 1:L
            for y in 1:L
                decision = rand(1:4)    # randomly pick neigbour to copy opinion from
                # why doesn't Julia natively support switch-case statement huh? -__-
                if decision == 1
                    # mod1() makes sure we dont go over the index range,
                    # insted we just go back to the start
                    network[x, y] = network[mod1(x - 1, L), y] # up
                elseif decision == 2
                    network[x, y] = network[mod1(x + 1, L), y] # down
                elseif decision == 3
                    network[x, y] = network[x, mod1(y - 1, L)] # left
                else
                    network[x, y] = network[x, mod1(y + 1, L)] # right
                end
            end
        end
        push!(yesses, sum(network)) # add number of ones / yes answers to the list                  
    end
    end_time = time()

    elapsed_time = end_time - start_time

    # end message using fancy ternary operator
    i == max_iter ? println("Model did not converge, max_iter=$i reached") : println("Model converged after $i iterations")


    return (i, yesses, elapsed_time)
end

voter_model (generic function with 1 method)

In [24]:
(iteration, yesses, elapsed) = voter_model(L=100);

Model converged after 7764 iterations


# Zad 2.  
Powtarzamy dziesięciokrotnie symulacje mierząc średni czasy dojścia do konsensusu (dziesieć różnych punktów startowych). Wyznaczamy średnią z tych dziesięciu pomiarów i niepewność jej wyznaczenia.  
# Zad 3.  
Sprawdźmy czy i jak średni czas dojścia do konsensusu (z dziesięciu realizacji) zależy od rozmiaru układu L2? Przyjmijmy L= 10, 50, 100, 200.

# Sprawozdanie

Wprowadzenie (25 pkt.) Zdefiniowanie modelu, opisanie problemu.  
Wyniki (50 pkt.)  
    ad 1: kilka wizualizacji sieci w kolejnych krokach czasowych. Wykres zależności czasowej liczby kroków na „tak”.  
    ad 2: Średnia z czasów dojścia do konsensusu wraz z jej niepewnością (odchyleniem standardowym średniej).  
    ad 3: Wykres τ(L2).  
Wnioski (25 pkt.) Jakie zjawisko udało się zaobserwować? W jaki sposób zmienia się czas dochodzenia
do konsensusu z liczbą aktorów?